**Sistemas de Recomendação**

Problema de aprendizagem: desenvolver um sistema básico de recomendação de filmes

Vamos nos concentrar em fornecer um sistema de recomendação básico, sugerindo itens que são mais semelhantes a um outro item específico, neste caso, filmes.

Neste projeto iremos utilizar dois datasets: 
- u.data
- Movie_Id_Titles

**Tenha em mente que este não é um sistema de recomendação verdadeiramente robusto. Ele apenas informa quais filmes/itens são mais semelhantes à sua escolha de filme.**

Importando as primeiras bibliotecas:

In [1]:
import numpy as np
import pandas as pd

Importando os datasets

In [6]:
# Importando o primeiro dataset(u.data) com as revisões de filmes dos usuários
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('/content/drive/MyDrive/Datasets/u.data', sep='\t', names=column_names)

In [ ]:
df.head()

In [ ]:
# Importando o segundo dataset(Movie_Id_Titles) 
# que contém os títulos dos mesmos filmes do primeiro dataset
df2 = pd.read_csv('/content/drive/MyDrive/Datasets/Movie_Id_Titles')
df2.head()

**Merge**

É uma função que funde Dataframes ou Séries em pandas.

No nosso problema de aprendizado, temos dois dataframes com uma coluna em comum. Iremos juntá-las no primeiro dataframe.

In [9]:
# unindo os dois datasets com base em uma coluna em comum: item_id
# parâmetros utilizados: (primeiro dataframe, segundo dataframe, coluna em comum pela qual faremos a união)
df = pd.merge(df,df2,on='item_id')

In [ ]:
# Agora temos a coluna com os títulos de cada filme no nosso primeiro dataframe
df.head()

**Análise exploratória dos dados**

Agora iremos visualizar como os nossos dados estão distribuídos

Importando as bibliotecas para visualização de dados

In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white') #a função set_style irá definir o estilo dos plots do Seaborn

**Groupby**

A função groupby () é usada para dividir os dados em grupos com base em alguns critérios.

Vamos criar um dataframe de avaliações contendo a média e o número de avaliações

In [ ]:
# groupby('title') : agrupando pela coluna title
# ['rating].mean() : pegando a média de classificações
# .sort_values (ascending=False) : ordenar os valores em ordem decrescente
df.groupby('title')['rating'].mean().sort_values(ascending=False).head()
# Agora temos a média de avaliação que cada filme recebeu

In [ ]:
# Agora vamos ver os filmes com mais avaliações
# ['rating'].count() : contagem das avaliações
df.groupby('title')['rating'].count().sort_values(ascending=False).head()

O groupy nos retornou os valores em uma Série. 

Para melhorar a visualização, vamos colocar isso tudo em um dataframe.

In [ ]:
ratings = pd.DataFrame(df.groupby('title')['rating'].mean())
ratings.head()

**Tenha em mente que a média de avaliação de um filme depende de quantas pessoas o avaliaram. Se um filme tem uma média 5.0 de avaliação, isso não significa necessariamente que ele está entre os mais bem avaliados. Ele pode ter recebido apenas uma avaliação de 5.0** 

Para ter uma idéia de quantas pessoas avaliaram cada filme, vamos criar uma coluna com a quantidade de avaliações:

In [ ]:
# criando uma coluna num of ratings no dataframe ratings
# a coluna irá receber o groupby com a contagem das avaliações em cada filme
ratings['num of ratings'] = pd.DataFrame(df.groupby('title')['rating'].count())
ratings.head()

**Agora vamos criar um histograma para visualizar os dados**

In [ ]:
# figsize (largura, altura)
# “bins” é o número de intervalos que o histograma considera para criar as barras
plt.figure(figsize=(10,4))
ratings['num of ratings'].hist(bins=70)

**Conclusão:** poucos filmes receberam muitas avaliações. A maioria tem 0 ou 1 avaliação.

Levando em consideração que a maioria das pessoas apenas assistem os filmes que são sucessos de bilheteria, faz sentido afirmar que são esses que receberam mais revisões no nosso dataset.


**Vamos criar um jointplot**

In [ ]:
# joinplot : plota um gráfico de duas variáveis com gráficos bivariados e univariados.
# alpha=0.5 : transparência dos pontos
sns.jointplot(x='rating',y='num of ratings',data=ratings,alpha=0.5)

**Conclusão:** à medida que um filme recebe mais avaliações, é mais provável que ele obtenha uma classificação mais elevada. 

Quanto melhor o filme, mais pessoas assistem e, consequentemente, avaliam.

Agora que temos uma ideia geral da aparência dos dados, vamos prosseguir para a **criação de um sistema de recomendação simples** 

Vamos criar uma matriz que tenha os ids do usuário em um eixo e o título do filme em outro eixo. 

Cada célula corresponde à avaliação que o usuário deu ao filme. 

Observe que haverá muitos valores NaN, porque a maioria das pessoas não viu a maioria dos filmes. 

In [ ]:
# pivot_table: cria uma tabela dinâmica em estilo de planilha como um DataFrame
moviemat = df.pivot_table(index='user_id',columns='title',values='rating')
moviemat.head()

In [ ]:
# 10 filmes mais avaliados
ratings.sort_values('num of ratings',ascending=False).head(10)

Vamos escolher dois filmes: Star Wars(ficção científica)e Liar Liar (comédia) para avaliar.

In [20]:
# vamos pegar as avaliações dos usuários para esses dois filmes: 
starwars_user_ratings = moviemat['Star Wars (1977)']
liarliar_user_ratings = moviemat['Liar Liar (1997)']

In [ ]:
# verificando 
starwars_user_ratings.head()
# agora temos uma Série com os ids dos usuários e a classificação que eles deram ao filme
# NaN : não assistiu ou não avaliou

**Corrwith**

Podemos usar o método corrwith () para obter correlações entre duas séries de pandas

In [ ]:
similar_to_starwars = moviemat.corrwith(starwars_user_ratings)
similar_to_liarliar = moviemat.corrwith(liarliar_user_ratings)

In [ ]:
similar_to_starwars.head(10)

Vamos limpar isso removendo os valores NaN e usar um DataFrame em vez de uma série: 

In [ ]:
# .dropna(inplace=True) : faz a operação inplace e retorna None. 
corr_starwars = pd.DataFrame(similar_to_starwars,columns=['Correlation'])
corr_starwars.dropna(inplace=True)
corr_starwars.head(10)

Se ordenarmos o dataframe por correlação, devemos obter os filmes mais semelhantes.

No entanto, obtemos alguns resultados que não fazem sentido. Isso ocorre porque há muitos filmes assistidos apenas uma vez por usuários que também assistiram a Star Wars (foi o filme mais popular). 

In [ ]:
corr_starwars.sort_values('Correlation',ascending=False).head(10)

Vamos corrigir isso filtrando os filmes com mais de 100 avaliações 

In [ ]:
# usando o join para juntar o número de avaliações ao meu dataframe com as correlações de star wars
corr_starwars = corr_starwars.join(ratings['num of ratings'])
corr_starwars.head()

Ordenando os valores, observamos como os títulos fazem muito mais sentido: 


In [ ]:
# filtrando os filmes que receberam mais de 100 avaliações 
# ordenando os valores pela coluna Correlation em ordem decrescente
corr_starwars[corr_starwars['num of ratings']>100].sort_values('Correlation',ascending=False).head()

**Agora faremos o mesmo para a comédia Liar Liar** 

In [ ]:
corr_liarliar = pd.DataFrame(similar_to_liarliar,columns=['Correlation'])
corr_liarliar.dropna(inplace=True)
corr_liarliar = corr_liarliar.join(ratings['num of ratings'])
corr_liarliar[corr_liarliar['num of ratings']>100].sort_values('Correlation',ascending=False).head()

Fim! :)